In [12]:
# Bibliotecas
import pandas as pd
import os
from sqlalchemy import create_engine, inspect
from io import BytesIO
import requests

In [5]:
# Funções
## Função para editar a tabela de Receita
def edit_sheet(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  
    df.columns = df.columns.str.replace('\n', '_')
    df.columns = df.columns.str.replace(' ', '_')
    return df

## Função para gerar um DBML a partir dos arquivos DB
def sql_to_dbml(engine):
    inspector = inspect(engine)
    dbml = ""

    for table_name in inspector.get_table_names():
        dbml += f"Table {table_name} {{\n"

        # Pega as colunas
        for column in inspector.get_columns(table_name):
            name = column['name']
            dtype = column['type']
            nullable = column['nullable']
            default = column['default']
            is_primary = name in [pk['constrained_columns'][0] for pk in inspector.get_pk_constraint(table_name).get('constrained_columns', [])]

            dbml += f"  {name} {dtype}"

            if is_primary:
                dbml += " [pk]"
            if not nullable:
                dbml += " [not null]"
            if default is not None:
                dbml += f" [default: {default}]"
            dbml += "\n"

        dbml += "}\n\n"

    return dbml

In [6]:
# Dicionários
# com os caminhos do arquivos e atributos da tabela

base1 = {
    "sheet_id": '1cucnW4yVosO5n5BFgwXYv6rVy8yj6NTasM83RTCMOug',
    "gid_receita": "373473243",
    "gid_despesas": "1859279676",
    "encoding": "latin1",
    "db_path": os.path.abspath("../database/bronze/base1.db"),
    "dbml_path": os.path.abspath("../modelagem/base1.dbml"),
}

base2 = {
    "nome_tabela": "vendas_clientes_consigcar",
    "file_path": os.path.abspath("..\\data\\Base 2 - Vendas Clientes ConsigCar(Vendas).csv"),
    "db_path": os.path.abspath("../database/bronze/base2.db"),
    "dbml_path": os.path.abspath("../modelagem/base2.dbml"),
    "encoding": "latin1",
}

## Extraindo Base 1 (Vendas)

### Aba Receita

In [8]:
#Lendo e armazenando em um dataframe as tabelas
df_Receita = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{base1['sheet_id']}/export?format=csv&gid={base1['gid_receita']}", index_col=False)
df_Receita = edit_sheet(df_Receita)

In [10]:
# Inciando tratamento
# Exclusão de linhas sem dados nas colunas Data e Mês
df_Receita = df_Receita.dropna(subset=["Data", "Mes"])

# Transformação da coluna data
df_Receita['Data'] = pd.to_datetime(df_Receita['Data'], dayfirst = True).dt.date

# Transformação das colunas Mês e Ano para inteiro
df_Receita["Mes"] = df_Receita["Mes"].astype(int)
df_Receita["Ano"] = df_Receita["Ano"].astype(int)

# Separação das colunas da Alucar e Consigcar
df_Receita_Alucar = df_Receita.iloc[:, 0:5] \
                    .reset_index(drop = True)

df_Receita_ConsigCar = df_Receita.iloc[:, [1,6]] \
                        .dropna(subset=["Valor"]) \
                        .reset_index(drop = True)

# Separação do dataframe da Receita da Alucar e estimativa da Alucar
df_Receita_Alucar_estimativa = df_Receita_Alucar[df_Receita_Alucar["Nome_(Alucar)"] == 'Estimativa'] \
                                .reset_index(drop = True)

df_Receita_Alucar = df_Receita_Alucar[df_Receita_Alucar["Nome_(Alucar)"] != 'Estimativa']

# Cria conexão com banco de dados SQLite
engine = create_engine(f"sqlite:///{base1['db_path']}")

# Insere as tabelas no banco
df_Receita_Alucar.to_sql("00_vendas_clientes_alucar", con=engine, if_exists="replace", index=False)
df_Receita_Alucar_estimativa.to_sql("00_vendas_clientes_alucar_estimativa", con=engine, if_exists="replace", index=False)
df_Receita_ConsigCar.to_sql("00_receita_pagseguro_consigcar", con=engine, if_exists="replace", index=False)


4

### Aba Despesas 

In [ ]:
# Lendo e armazenando em um dataframe as tabelas
df_Despesas = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{base1["sheet_id"]}/export?format=csv&gid={base1["gid_despesas"]}", index_col=False)

# Excluindo colunas desnecessárias
df_Despesas.columns = df_Despesas.iloc[0]
df_Despesas = df_Despesas[1:]
df_Despesas.reset_index(drop = True, inplace = True)

#Separando Despesas da Alucar e ConsigCar

# Encontrar o índice da linha que contém TOTAL na coluna Despesa
index_total = df_Despesas[df_Despesas['DESPESAS'].str.contains("TOTAL", na=False)].index.min()

# Selecionar todas as linhas até a linha anterior à que contém TOTAL
df_Despesas_Alucar = df_Despesas.iloc[:index_total].copy()
df_Despesas_Alucar = df_Despesas_Alucar.reset_index(drop = True) \
                                       .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")

# Selecionar as linhas referentes a ConsigCar
df_Despesas_ConsigCar = df_Despesas.iloc[index_total+4:-1] \
                        .copy().reset_index(drop = True) \
                        .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")

# Cria conexão com banco de dados SQLite
engine = create_engine(f"sqlite:///{base1["db_path"]}")

# Insere as tabelas no banco
df_Despesas_Alucar.to_sql("00_despesas_alucar", con=engine, if_exists="replace", index=False)
df_Despesas_ConsigCar.to_sql("00_despesas_consigcar", con=engine, if_exists="replace", index=False)


156

In [107]:
# Transformar o banco de dados em DBML para acompanhamento da modelagem
dbml = sql_to_dbml(engine)

with open(base1["dbml_path"], "w", encoding="utf-8") as f:
    f.write(dbml)

### Extraindo Base 2

In [ ]:
# Lendo e aramazenando em um dataframe as tabelas
url = 'https://empregadados-my.sharepoint.com/personal/bianca_empregadados_com_br/_layouts/15/download.aspx?share=EZYutqfo5ldNhDw2lMYRxrIBnpPI6c7OTjBBS_F5yz860Q'

response = requests.get(url)
df = pd.read_excel(BytesIO(response.content))
#df = pd.read_csv(base2["file_path"], encoding=base2["encoding"])

# Remove 'R$', pontos e converte a vírgula decimal para ponto
df['Valor parcela'] = df['Valor parcela'].replace({'R\$': '', '\.': '', ',': '.'}, regex=True).astype(float)

# Converte a coluna de data e remove a hora
df['Data do Pagamento'] = pd.to_datetime(df['Data do Pagamento']).dt.date

# Cria conexão com banco de dados SQLite
engine = create_engine(f"sqlite:///{base2['db_path']}")

# Insere a tabela no banco
df.to_sql("00_vendas_clientes_consigcar", con=engine, if_exists="replace", index=False)

207

In [109]:
# Transformar o banco de dados em DBML para acompanhamento da modelagem
dbml = sql_to_dbml(engine)

with open(base2["dbml_path"], "w", encoding="utf-8") as f:
    f.write(dbml)